# SeeTrue Competition: Code for DeepFakes
_Code Template from DSTA_

In [ ]:
#@markdown ##**Connect to a GPU Runtime** 
#@markdown GPUs speed up the training of deep learning models by enabling parallel computations. Connect to a GPU runtime to speed up the running of this notebook.

def display_gpu():
  """
  Print the GPU runtime that you are connected to.
  """
  return !nvidia-smi

display_gpu()

In [ ]:
#@markdown ##**Install required libraries**

models_dir = "/content/models"
images_dir = "/content/images"
results_dir = "/content/results"
character_img = "/content/character_img.png"
trimmed_character_img = "/content/trimmed_character_img.png"
driving_video = "/content/driving_video.mp4"
animated_video_no_audio = f"{results_dir}/animated_video_no_audio.mp4"
animated_video = f"{results_dir}/animated_video.mp4"
final_video = f"{results_dir}/final_video.mp4"
 

%cd "/content"  
print("Downloading Packages...")

from pathlib import Path
Path(models_dir).mkdir(parents=True,exist_ok=True)
Path(images_dir).mkdir(parents=True,exist_ok=True)
Path(results_dir).mkdir(parents=True,exist_ok=True)

# First-Order-Model
!git clone "https://github.com/seetrueinfo/first-order-model.git"
!wget -nc "https://raw.githubusercontent.com/opencv/opencv/master/data/haarcascades/haarcascade_frontalface_alt2.xml" -O "/content/first-order-model/haarcascade_frontalface_alt2.xml" &> /dev/null

# JojoGAN
!git clone https://github.com/seetrueinfo/JoJoGAN.git
!wget -nc https://github.com/ninja-build/ninja/releases/download/v1.8.2/ninja-linux.zip
!sudo unzip -n ninja-linux.zip -d /usr/local/bin/
!sudo update-alternatives --install /usr/bin/ninja ninja /usr/local/bin/ninja 1 --force
!rm "/content/ninja-linux.zip"

print("Installing required libraries")
!pip install -r first-order-model/requirements.txt &> /dev/null 
!pip install ffmpeg scikit-video &> /dev/null 

print("Succesfully Finished Installing Libraries")

In [ ]:
#@markdown ##**Import libraries**
#@markdown A program must import libraries before using them. 

#@markdown In this cell, we import all libraries to be used and provide some helper functions that will be used throughout this notebook.
#@markdown This may take a while.

print("Loading Libraries and helper functions...")

%load_ext autoreload
%autoreload 2
%matplotlib inline

import sys
sys.path.append('/content/JoJoGAN')
import shutil 
shutil.rmtree('/content/sample_data',ignore_errors=True)
from util import *  
from torchvision import utils 
import torch 
from PIL import Image  
from model import *

import sys
import os
import numpy as np
import ipywidgets as widgets
from io import StringIO
from IPython import get_ipython
from scipy.io import wavfile
import skvideo.io  
from google.colab import files
from skimage import img_as_ubyte

# First-order-model
import imageio
import cv2 
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from skimage.transform import resize
from numpy.core import memmap
import imutils

# Record and display video 
from google.colab.output import eval_js
from IPython.display import Javascript, display, HTML, Image as display_image
from base64 import b64decode, b64encode


def getLocalFiles():
    """
    Helper function to upload files from your laptop to this Colab notebook.
    Note that files are not persistent! (i.e. if you restart runtime, all your
    uploads will be lost)
    """
    uploaded = files.upload()
    filename = next(iter(uploaded))
    return filename


def record_video(filename):
    """
    Helper function to record video in Colab.
    """
    js=Javascript("""
        async function recordVideo() {
        const options = { mimeType: "video/webm; codecs=vp9" };
        const div = document.createElement('div');
        const capture = document.createElement('button');
        const stopCapture = document.createElement("button");
        
        capture.textContent = "Start Recording";
        capture.style.background = "orange";
        capture.style.color = "white";

        stopCapture.textContent = "Stop Recording";
        stopCapture.style.background = "red";
        stopCapture.style.color = "white";
        div.appendChild(capture);

        const video = document.createElement('video');
        const recordingVid = document.createElement("video");
        video.style.display = 'block';

        const stream = await navigator.mediaDevices.getUserMedia({audio:true, video: true});
        
        let recorder = new MediaRecorder(stream, options);
        document.body.appendChild(div);
        div.appendChild(video);

        video.srcObject = stream;
        video.muted = true;

        await video.play();

        google.colab.output.setIframeHeight(document.documentElement.scrollHeight, true);

        await new Promise((resolve) => {
            capture.onclick = resolve;
        });
        recorder.start();
        capture.replaceWith(stopCapture);

        await new Promise((resolve) => stopCapture.onclick = resolve);
        recorder.stop();
        let recData = await new Promise((resolve) => recorder.ondataavailable = resolve);
        let arrBuff = await recData.data.arrayBuffer();
        
        // stop the stream and remove the video element
        stream.getVideoTracks()[0].stop();
        div.remove();

        let binaryString = "";
        let bytes = new Uint8Array(arrBuff);
        bytes.forEach((byte) => {
            binaryString += String.fromCharCode(byte);
        })
        return btoa(binaryString);
        }
    """
    )
    try:
        display(js)
        data=eval_js('recordVideo({})')
        binary=b64decode(data)
        with open(filename,"wb") as video_file:
            video_file.write(binary)
            print(f"Finished recording video at:{filename}")
    except Exception as err:
        print(str(err))


def display_video(vid_filename):
    """
    Display the video in colab. 
    """
    # display result
    mp4 = open(vid_filename,'rb').read()
    data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
    return HTML("""
    <video width=400 controls>
        <source src="%s" type="video/mp4">
    </video>
    """ % data_url)

print("Done!")

In [ ]:
#@markdown ##**Download Models and images** 

drive_ids = {  
    "dlibshape_predictor_68_face_landmarks.dat":"https://www.dropbox.com/s/2a0arq1px988ry4/dlibshape_predictor_68_face_landmarks.dat?dl=0",
    "bella_einstein.mp4":"https://www.dropbox.com/s/ugynax1n3vt049w/bella_einstein.mp4?dl=0",
    "driving_video.mp4":"https://www.dropbox.com/s/j2xes309icr1kor/driving_video.mp4?dl=0", 
    "logo_small.png":"https://www.dropbox.com/s/pkczqq59s5vi1kt/logo_small.png?dl=0", 
    "einstein.png": "https://www.dropbox.com/s/eitp4rlh516b7lx/einstein.png?dl=0",
    "lincoln.png":"https://www.dropbox.com/s/efr8y186cwbuka0/lincoln.png?dl=0",
    "nietzsche.png":"https://www.dropbox.com/s/zbxyuwvooy3oh90/nietzsche.png?dl=0",
    "sokrates.png":"https://www.dropbox.com/s/6ehlgqeacs2a2nt/sokrates.png?dl=0",
    "van_gogh.png":"https://www.dropbox.com/s/i4jqk3pu8u8gxj1/van_gogh.png?dl=0",
    "mona_lisa.png": "https://www.dropbox.com/s/ewkmhyl6o8znp8w/mona_lisa.png?dl=0",
    "vox-cpk.pth.tar" : "https://www.dropbox.com/s/ih36quf8vlwagxh/vox-cpk.pth.tar?dl=0",
    "stylegan2-ffhq-config-f.pt": "https://www.dropbox.com/s/4lhdbzmjanvaei3/stylegan2-ffhq-config-f.pt?dl=0",
    "e4e_ffhq_encode.pt": "https://www.dropbox.com/s/3u1ewa19m3ovvdj/e4e_ffhq_encode.pt?dl=0", 
    "arcane_caitlyn.pt": "https://www.dropbox.com/s/3nzkazv9lmssl4z/arcane_caitlyn.pt?dl=0",
    "arcane_caitlyn_preserve_color.pt": "https://www.dropbox.com/s/rl0726jpw1fwiav/arcane_caitlyn_preserve_color.pt?dl=0",
    "arcane_jinx_preserve_color.pt": "https://www.dropbox.com/s/dqw421ahlv1aac9/arcane_jinx_preserve_color.pt?dl=0",
    "arcane_jinx.pt": "https://www.dropbox.com/s/ri7u36vwith0k3k/arcane_jinx.pt?dl=0",
    "arcane_multi_preserve_color.pt": "https://www.dropbox.com/s/732gtuvhofyb0t5/arcane_multi_preserve_color.pt?dl=0",
    "arcane_multi.pt": "https://www.dropbox.com/s/fb54ml11u1tyrab/arcane_multi.pt?dl=0",
    "sketch_multi.pt": "https://www.dropbox.com/s/296weom7h43uedj/sketch_multi.pt?dl=0",
    "disney.pt": "https://www.dropbox.com/s/lqt1fcztw6ryp06/disney.pt?dl=0",
    "disney_preserve_color.pt": "https://www.dropbox.com/s/v4kg3tq276ibrjh/disney_preserve_color.pt?dl=0",
    "jojo.pt": "https://www.dropbox.com/s/clx9xvc3pnsd7nl/jojo.pt?dl=0",
    "jojo_preserve_color.pt": "https://www.dropbox.com/s/5mtgon0dldboisf/jojo_preserve_color.pt?dl=0",
    "jojo_yasuho.pt": "https://www.dropbox.com/s/9ckqmpf2kp6x892/jojo_yasuho.pt?dl=0",
    "jojo_yasuho_preserve_color.pt": "https://www.dropbox.com/s/cezm4mbjco97wcx/jojo_yasuho_preserve_color.pt?dl=0",
    "art.pt": "https://www.dropbox.com/s/cj3e77xe6xgxwp5/art.pt?dl=0",
}

class Downloader(object):  
    def download_file(self, file_name, folder, use_gdown=False, always_download=False):
        file_dst = os.path.join(folder, file_name)
        file_id = drive_ids[file_name]
        if not os.path.exists(file_dst) or always_download:
            print(f'Downloading {file_name}') 
            if use_gdown:
              !gdown $file_id -O $file_dst
            else:
              !wget -nc $file_id -O $file_dst
            return False
        else:
          return True

downloader = Downloader()
downloader.download_file("dlibshape_predictor_68_face_landmarks.dat",models_dir)

Step 1: Data Preparation

In [ ]:
#@markdown ##**Choose Image**

#@markdown Choose the character that you want to animate and then run the cell to confirm your selection. 

#@markdown ![](https://drive.google.com/uc?export=view&id=14hmm9YSvhVGerqRtou3yDGnyzMl_05ry)

%cd "/content"  
character = 'Upload Your Own' #@param ["Van Gogh", "Mona Lisa", "Einstein", "Lincoln", "Nietzsche", "Sokrates", "Upload Your Own"]
print(f"{character} selected.")

if character == "Upload Your Own":
  character_selected = f"/content/{getLocalFiles()}"
  if character_selected.endswith(".jpg"):
    Image.open(character_selected).save(character_img)

else:
  character = character.lower().replace(" ", "_") # make lowercase and remove spacing
  downloader.download_file(f'{character}.png',images_dir) 
  character_selected = f"{images_dir}/{character}.png"

!cp $character_selected $character_img

my_w = None

from e4e_projection import projection as e4e_projection ###

downloader.download_file('e4e_ffhq_encode.pt',models_dir) ###
aligned_face = align_face(character_img)   ###
if aligned_face: ###    
  name = strip_path_extension(character_img)+'.pt' ###
  my_w = e4e_projection(aligned_face, name, 'cuda').unsqueeze(0)   ###

display_image(character_img, width=400, height=400)


In [ ]:
#@markdown ##**Upload/Record Driving Video**
#@markdown Your driving video should contain the action that want your character to copy.

#@markdown **Important to note that the bigger the face the better the output!**

#@markdown Video requirements: 
#@markdown - ~30 frames per second (otherwise your generated character will move faster/slower)
#@markdown - Contain one face in every frame
#@markdown - ~30 seconds (Colab might run out of RAM if you upload/record a video that is too long)
#@markdown - .MP4 format 

#@markdown Either record the video using your webcam or upload the video from a file (.mp4). Note that the record function only works on Google Chrome/Microsoft Edge browsers. 
record_or_upload = "Record" #@param ["Bella Poarch", "Record", "Upload (.mp4)"]

if record_or_upload == "Bella Poarch":
  downloader.download_file('driving_video.mp4',folder="/content",always_download=True)

elif record_or_upload == "Record":
    print("Please record the video you wish to drive the animation with. Remember to enable your camera and microphone in Chrome:\n")
    button = widgets.Button(description="Record Your Video") 
    record_video(driving_video)
else:
    print("Please upload the video you wish to drive the animation with:\n")
    uploaded = f"/content/{getLocalFiles()}"
    if uploaded != driving_video:
      os.rename(uploaded,driving_video)


In [ ]:
#@markdown ##**Verify Your Driving Video**
#@markdown Ensure that you have uploaded the right video. If you are not satisfied with how it turned out, feel free to return to the previous cell to re-upload/record. 

display_video(driving_video)

Step 2: Style Character

In [ ]:
def style_image(character_img):
    """
    Stylize `character_img` (e.g. using JoJoGan) and return the stylized output. 
    """
    pretrained = 'art' 
    preserve_color = False

    if preserve_color:
        ckpt = f'{pretrained}_preserve_color.pt'
    else:
        ckpt = f'{pretrained}.pt'

    # load base version if preserve_color version not available
    try:
        downloader.download_file(ckpt)
    except:
        ckpt = f'{pretrained}.pt'
        downloader.download_file(ckpt, models_dir)

    generator = Generator(1024, 512, 8, 2).to('cuda')
    ckpt = torch.load(os.path.join(models_dir, ckpt),
                      map_location=lambda storage, loc: storage)
    generator.load_state_dict(ckpt["g"], strict=False)

    with torch.no_grad():
        generator.eval()
        my_sample = generator(my_w, input_is_latent=True)

    return my_sample

output = style_image(character_img)

In [ ]:
# save the output to the file character_img
save_image(utils.make_grid(output, normalize=True, range=(-1, 1)), 
           save_location=character_img)
# display the generated image
display_image(character_img, width=400, height=400)

Step 3: Animate it

In [ ]:
def trim_img(img_src, img_out=trimmed_character_img):
    """
    Trim `img_src` to 400 x 400 pixels centered on a face. The trimmed image will 
    be saved to trimmed_character_img unless otherwise specified. 
    """
    
    # read the input image
    img = cv2.imread(img_src)
    img = imutils.resize(img, width=400)

    # convert into grayscale
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # detect where the face is located
    face_cascade = cv2.CascadeClassifier(
        '/content/first-order-model/haarcascade_frontalface_alt2.xml')
    faces = face_cascade.detectMultiScale(gray, 1.1, 4)
    try:
        for (x, y, w, h) in faces:
            extention = 40
            faces = img[y-extention:y + h+extention,
                        x-extention:x + w + extention]
            cv2.imwrite(img_out, faces)
    except:
        print("Error: Face takes too much space on image. Try a different image, or trim it yourself to 400x400.")
    return img_out

def animate_video(img_filename, vid_filename, vid_out=animated_video):
    """
    Animate `img_filename` using the video in `vid_filename`. Motions from the 
    driving video (`vid_filename`) is transfered to object in the static image.
    Return the animated frames.  
    """
    # preprocess image to be 400x400
    trim_img(img_filename)
    downloader.download_file('vox-cpk.pth.tar', models_dir)

    %cd /content/first-order-model/

    from demo import make_animation
    from demo import load_checkpoints
    from skimage import img_as_ubyte

    # load the image to be animated
    source_image = imageio.imread(trimmed_character_img)

    # read the driving video at 30 fps
    driving_video = skvideo.io.vread(vid_filename, inputdict={'-r': "30"})

    # resize image and video to 256x256
    source_image = resize(source_image, (256, 256))[..., :3]
    driving_video = [resize(frame, (256, 256))[..., :3]
                     for frame in driving_video]

    # Load first order model to transfer motion
    generator, kp_detector = load_checkpoints(
        config_path='config/vox-256.yaml', checkpoint_path=f"{models_dir}/vox-cpk.pth.tar")

    # animate source image
    predictions = make_animation(source_image, driving_video, generator, kp_detector, relative=True,
                                 adapt_movement_scale=False)

    return predictions

animated_frames = animate_video(character_img, driving_video)

In [ ]:
# remove video if previously generated
if os.path.exists(animated_video_no_audio):
    os.remove(animated_video_no_audio)
    
# save resulting video as file named`animated_video_no_audio`
imageio.mimsave(animated_video_no_audio, 
                [img_as_ubyte(frame) for frame in animated_frames], fps=30)

# add audio back to video 
!ffmpeg -y -i $animated_video_no_audio -i $driving_video -c copy -map 0:v:0 -map 1:a:0 -strict -2 $animated_video

# add the seetrue logo 
downloader.download_file('logo_small.png',images_dir)    
!ffmpeg -y -i $animated_video -i /content/images/logo_small.png -filter_complex "overlay=0:H-h" $final_video

# display the generated video 
display_video(final_video) 

Step 4: Get Creative

In [ ]:
display_video(final_video)